In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from scipy.optimize import minimize
import os

# 1. 데이터 로드
DATA_PATH = "../../data/raw/"
OOF_PATH = "./oof_data/"

# 정답지 로드
train_data = pd.read_csv(f'{DATA_PATH}train.csv')
train_data = train_data.drop(train_data[train_data.familysize > 50].index)
y_true = (2 - train_data['voted'].to_numpy()).astype(np.float32)

# OOF 파일 로드 (파일명은 실제 생성된 이름과 일치해야 합니다)
oof_m1 = np.load(f"{OOF_PATH}exp11_sota_AUC_0.77212.npy")
oof_m2 = np.load(f"{OOF_PATH}exp12_auc_AUC_0.77233.npy")
oof_m4 = np.load(f"{OOF_PATH}exp15_snn_AUC_0.77330.npy")

# 2. 단순 가중치 점수 확인 (Exp 16에서 썼던 6:3:1 조합)
simple_blend = (oof_m1 * 0.6) + (oof_m2 * 0.3) + (oof_m4 * 0.1)
simple_auc = roc_auc_score(y_true, simple_blend)
print(f"--- Exp 16 (6:3:1) Local AUC: {simple_auc:.5f} ---")

# 3. 수학적 가중치 최적화 (Constrained Optimization)
def auc_func(weights):
    # 가중치 합이 1이 되도록 정규화
    w1, w2, w3 = weights
    blend = (oof_m1 * w1) + (oof_m2 * w2) + (oof_m4 * w3)
    return -roc_auc_score(y_true, blend) # 최소화 함수이므로 마이너스

# 초기값 및 제약 조건 (가중치 합 = 1, 각 가중치는 0~1 사이)
initial_weights = [0.33, 0.33, 0.34]
bounds = [(0, 1), (0, 1), (0, 1)]
constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}

result = minimize(auc_func, initial_weights, method='SLSQP', 
                  bounds=bounds, constraints=constraints)

best_w = result.x
best_auc = -result.fun

print("\n" + "="*40)
print(f"로컬 최적 가중치 발견!")
print(f"m1 (exp11_sota): {best_w[0]:.4f}")
print(f"m2 (exp12_auc) : {best_w[1]:.4f}")
print(f"m4 (exp15_snn) : {best_w[2]:.4f}")
print(f"최적화된 로컬 AUC : {best_auc:.5f}")
print("="*40)

--- Exp 16 (6:3:1) Local AUC: 0.77293 ---

로컬 최적 가중치 발견!
m1 (exp11_sota): 0.3300
m2 (exp12_auc) : 0.3299
m4 (exp15_snn) : 0.3400
최적화된 로컬 AUC : 0.77351
